# Chapter 08. 선형회귀의 심화
## 03. 사이킷런을 이용한 선형회귀
### 1 - 사이킷런과 선형회귀 관련 함수
- 여러 선형회귀 알고리즘을 사이킷런(scikit-learn)을 활용하여 적용하고자 함
    - 사이킷런(scikit-learn) : 대표적인 머신러닝 라이브러리
- 사이킷런의 선형회귀 관련 함수
    - LinearRegression
        - 가장 기본적인 선형회귀 알고리즘
        - SGD가 아닌 최소자승법으로 계산
    - Lasso
        - L1 손실을 활용한 라쏘 알고리즘 사용
        - 최소자승법으로 계산
    - Ridge
        - L2 손실을 활용한 리지 알고리즘 사용
        - 최소자승법으로 계산
    - SGDRegressor
        - 확률적 경사하강법을 사용한 회귀 모델을 만듦
        - SGD에서 비용함수만을 변경하여 모든 함수를 지원
            - 필요한 하이퍼 매개변수를 설정해야 함
        - SGD 활용

- 기본적으로 사이킷런에서는 sklearn.linear_model 안에 관련된 여러 클래스를 제공
- 앞서 최소자승법과 SGD의 두 가지 알고리즘 위에 기본적인 선형회귀와 리지, 라쏘 회귀가 구현된다고 배웠음
    - 사이킷런도 이와 동일하게 기본적인 최소자승법 기반의 알고리즘은 세 개의 클래스 제공
    - SGD 기반의 알고리즘은 하나의 알고리즘을 제공하여 사용하도록 지원

### 2 - 사이킷런을 활용하여 선형회귀 구현하기
- 이번에 다룰 데이터셋은 'boston housing prices'데이터셋

#### 2.1 데이터 확보하기
- 데이터를 분석하기 위해 사이킷런을 통해 데이터를 확보해야 함
    - 데이터 처리에 중점을 맞추기 위해 데이터를 확보하고자 함
- sklearn.datasets 라이브러리에 있는 load_boston 모듈을 사용하여 데이터 추출
    - 해당 데이터를 추출하면 딕셔너리 타입의 객체를 반환
        - x 데이터셋에 해당하는 data
        - y 데이터셋에 해당하는 target
        - 모든 피쳐 정보를 담고 있는 feature_names
        - 해당 피쳐에 대한 설명이 있는 DESCR
        - 해당 데이터가 현재 컴퓨터 어디에 저장되어 있는지 알려주는 filename
        - 등이 있음

In [8]:
from sklearn.datasets import fetch_openml
# 사이킷런 1.2버전 이후로 load_boston 함수가 사라짐
    # 대신 fetch_openml 함수를 사용하여 boston 변수를 교재와 똑같이 만듦

import matplotlib.pyplot as plt
import numpy as np

boston = fetch_openml(name= 'boston')
boston.keys()

C:\Users\xodud\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\datasets\_openml.py:311: UserWarning: Multiple active versions of the dataset matching the name boston exist. Versions may be fundamentally different, returning version 1.
  warn(
C:\Users\xodud\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\datasets\_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [13]:
np.array(boston['data'])

array([[0.00632, 18.0, 2.31, ..., 15.3, 396.9, 4.98],
       [0.02731, 0.0, 7.07, ..., 17.8, 396.9, 9.14],
       [0.02729, 0.0, 7.07, ..., 17.8, 392.83, 4.03],
       ...,
       [0.06076, 0.0, 11.93, ..., 21.0, 396.9, 5.64],
       [0.10959, 0.0, 11.93, ..., 21.0, 393.45, 6.48],
       [0.04741, 0.0, 11.93, ..., 21.0, 396.9, 7.88]], dtype=object)

In [23]:
# 해당 데이터 중 x, y의 각 데이터셋 추출
# 아래 코드는 y_data의 경우 n * 1 형태로 변환하기 위해 매개변수 reshape 적용

x_data = np.array(boston["data"])
y_data = np.array(boston["target"]).reshape(np.array(boston["target"]).size, 1)
np.shape(y_data)

(506, 1)

#### 2.2 데이터 전처리하기
- 여기서는 간단하게 피쳐 스케일링(feature scaling)을 적용
- 사이킷런에서 제공하는 preprocessing 모듈 호출
    - 해당 모듈에 있는 MinMaxScaler 클래스 호출
        - 해당 클래스에서 fit 함수 사용, x_data 입력
            - 해당 데이터를 피쳐 스케일링할 수 있는 MinMaxScaler 객체가 생성됨

In [24]:
from sklearn import preprocessing

minmax_scale = preprocessing.MinMaxScaler(feature_range= (0, 5)).fit(x_data)
# feature_range는 최대 최솟값을 지정하는 매개변수
# 사이킷런의 다른 클래스처럼 MinMaxScaler 클래스의 경우도 fit 함수는 기본 틀이 만들어지는 함수
    # transform 함수는 적용을 하는 함수
x_scaled_data = minmax_scale.transform(x_data)
# 이미 만들어진 MinMaxScaler 클래스를 실제 데이터에 적용하여 스케일된 데이터 생성
    # 해당 결과를 출력하면 0과 5 사이의 값으로 스케일된 데이터 출력값 확인 가능

x_scaled_data[:3]

array([[0.00000000e+00, 9.00000000e-01, 3.39076246e-01, 0.00000000e+00,
        1.57407407e+00, 2.88752635e+00, 3.20803296e+00, 1.34601570e+00,
        0.00000000e+00, 1.04007634e+00, 1.43617021e+00, 5.00000000e+00,
        4.48399558e-01],
       [1.17961270e-03, 0.00000000e+00, 1.21151026e+00, 0.00000000e+00,
        8.64197531e-01, 2.73998850e+00, 3.91349125e+00, 1.74480990e+00,
        2.17391304e-01, 5.24809160e-01, 2.76595745e+00, 5.00000000e+00,
        1.02235099e+00],
       [1.17848872e-03, 0.00000000e+00, 1.21151026e+00, 0.00000000e+00,
        8.64197531e-01, 3.47192949e+00, 2.99691040e+00, 1.74480990e+00,
        2.17391304e-01, 5.24809160e-01, 2.76595745e+00, 4.94868627e+00,
        3.17328918e-01]])

#### 2.3 데이터 분류하기
- 학습을 위해서는 데이터를 훈련과 테스트 형태로 분류해야 함
- 이를 위해 model_selection 모듈에 있는 train_test_split 함수를 사용하여 데이터를 추출
    - train_test_split 함수는 X 데이터와 Y 데이터를 넣고 테스트 데이터셋의 비율을 test_size에 입력
        - X 데이터의 학습 데이터셋, X 데이터의 테스트 데이터셋, Y 데이터의 학습 데이터셋, Y 데이터의 테스트 데이터셋으로 랜덤하게 나누어 배분
- 데이터에 대한 각각의 데이터 개수는 다음 코드와 같음

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(x_scaled_data, y_data, test_size= 0.33)

np.shape(X_train), np.shape(X_test), np.shape(Y_train), np.shape(Y_test)

((339, 13), (167, 13), (339, 1), (167, 1))

#### 2.4 데이터 학습하기
- 학습을 위해서는 가장 먼저 학습에 사용할 알고리즘에 해당하는 모델의 클래스를 호출해야 함
    - 이전에 배웠던 기본적인 모델들을 사용할 수 있음
- 모델을 호출할 때 반드시 각 클래스들의 매개변수를 이해해야 함
    - 공통적으로 사용할 수 있는 매개변수
        - **fit_intercept** : 절편을 사용할 지 말지 선택하는 것
            - **True** or **False** 지정 가능
        - **normalize** : 학습할 때 값들을 정규화할지 말지에 대한 값
        - **copy_X** : 학습 시 데이터를 복사한 후 학습을 할 지에 대해 결정하는 매개변수
        - **n_jobs** : 연산을 위해 몇 개의 CPU를 사용할 지 결정하는 매개변수
    - 라쏘, 리지, SGD에는 페널티 값을 지정하는 **alpha**가 있음
        - SGD의 경우 직접 **penalty**함수 지정 가능 : **lambda**의 값을 **alpha**에 입력 가능